# Initial setup

Install Bambu and required packages:

In [ ]:
!add-apt-repository -y ppa:git-core/ppa
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git iverilog verilator
!wget https://release.bambuhls.eu:8080/appimage/bambu-showcase.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!ln -sf $PWD/bambu-*.AppImage /bin/tree-panda-gcc
!ln -sf $PWD/bambu-*.AppImage /bin/clang-12
!ln -sf $PWD/bambu-*.AppImage /bin/mlir-opt
!rm -rf PandA-bambu bambu-tutorial
!git clone --depth 1 --filter=blob:none --branch everest-school --sparse https://github.com/ferrandi/PandA-bambu.git
%cd PandA-bambu
!git sparse-checkout set documentation/everest_summer_school
%cd ..
!mv PandA-bambu/documentation/everest_summer_school/ bambu-tutorial

Check the installation:

In [ ]:
!bambu -h

In [ ]:
!mlir-opt --help

# Bambu inputs and outputs


Example C code in /content/bambu-tutorial/Exercise1/icrc.c

In [ ]:
%cd /content/bambu-tutorial/Exercise1
!bambu icrc.c --top-fname=icrc1 --simulator=VERILATOR --simulate --generate-tb=test_icrc1.xml -v2 --print-dot --pretty-print=a.c 2>&1 | tee icrc1.log

Inspect the generated files in the explorer tab on the left:

*   /content/bambu-tutorial/Exercise1/icrc1.v
*   /content/bambu-tutorial/Exercise1/simulate_icrc1.sh
*   /content/bambu-tutorial/Exercise1/synthesize_Synthesis_icrc1.sh
*   /content/bambu-tutorial/Exercise1/a.c



Visualize the FSM:

In [ ]:
from graphviz import Source
Source.from_file('HLS_output/dot/icrc1/HLS_STGraph.dot')

# Pointer to pointer synthesis example

Source code: /content/bambu-tutorial/01-introduction/Exercise2/tree.c

Search and insertion in a binary tree
 - Two data structures: stack and binary tree
 - Static memory allocators
 - Tail recursive functions
 - Use of pointer to pointers (some HLSs have problems)

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise2
!./bambu.sh

Inspect the generated files in the explorer tab on the left:

*   /content/bambu-tutorial/01-introduction/Exercise2/bambu.sh
*   /content/bambu-tutorial/01-introduction/Exercise2/search/profiling_results.txt


# HLS starting from .ll intermediate IR

Source code: /content/bambu-tutorial/01-introduction/Exercise3/Keccak.c

bambu script: /content/bambu-tutorial/01-introduction/Exercise3/bambu-clang12.sh

In [ ]:
%cd /content/bambu-tutorial/01-introduction/Exercise3/
!./bambu-clang12.sh

It processes 1600bits in 123 cycles at 400Mhz: 5203Mbit/s

Previous *old* results:
- Altera Cyclone II 7049LEs 152.0Mbit/s 114MHz (directory keccak_CycloneII_10ns)
- Altera Cyclone II 7434LEs 272.8Mbit/s 209MHz (directory keccak_CycloneII_4ns)
- Lattice ECP3 4038slices 151.7Mbit/s 114MHz (directory keccak_ECP3_9ns)
- Xilinx Virtex 5 1699slices 316.5Mbit/s 252MHz (directory keccak_V5_4ns)
- Xilinx Virtex 7 1887slices 529.8Mbit/s 406MHz (directory keccak_V7_2ns)


# ap_types and ac_types support

Synthesis of an acceleratore with input and output fifo ports.

In [ ]:
%mkdir -p /content/ap_ac_types
%cd /content/ap_ac_types

/content/ap_ac_types


In [ ]:
%%writefile ap_example.cpp
#include <algorithm>
#include "ap_int.h"
#define SIMULATION 1
#pragma HLS_interface x_fifo fifo
#pragma HLS_interface y_fifo fifo
#pragma HLS_interface res fifo
void gcd(ap_int<11>* x_fifo, ap_int<11>* y_fifo, ap_int<11>* res)
{
  do
  {
     ap_int<11> x = *x_fifo, y = *y_fifo;
     if( x < y )
          std::swap( x, y );

     while( y > 0 )
     {
          int f = x % y;
          x = y;
          y = f;
     }
     *res = x;
  } while(!SIMULATION);
}

Overwriting ap_example.cpp


In [ ]:
!bambu ap_example.cpp --simulate --no-clean --compiler=I386_CLANG12 --generate-interface=INFER --top-fname=gcd --simulator=VERILATOR

Check the output generated:
- /content/ap_ac_types/gcd.v

# MLIR affine dialect synthesis

## Prepare working directory

In [ ]:
%mkdir -p /content/mlir-simple
%cd /content/mlir-simple

/content/mlir-simple


## Matrix-multiply C=alpha.A.B+beta.C described using the MLIR Affine dialect.

In [ ]:
%%writefile gemm_32.mlir
func @gemm_32(%alpha: f32, %beta: f32, %C: memref<32x32xf32>, %A: memref<32x32xf32>, %B: memref<32x32xf32>) {
  affine.for %i = 0 to 32 {
    affine.for %j = 0 to 32 {
      %0 = affine.load %C[%i, %j] : memref<32x32xf32>
      %1 = arith.mulf %beta, %0 : f32
      affine.store %1, %C[%i, %j] : memref<32x32xf32>
      affine.for %k = 0 to 32 {
        %2 = affine.load %A[%i, %k] : memref<32x32xf32>
        %3 = affine.load %B[%k, %j] : memref<32x32xf32>
        %4 = affine.load %C[%i, %j] : memref<32x32xf32>
        %5 = arith.mulf %alpha, %2 : f32
        %6 = arith.mulf %5, %3 : f32
        %7 = arith.addf %4, %6 : f32
        affine.store %7, %C[%i, %j] : memref<32x32xf32>
      }
    }
  }
  return
}

Writing gemm_32.mlir


## Converting the MLIR specification into an LLVM IR using mlir-opt and mlir-translate. We obtain:

In [ ]:
%%writefile gemm_32.ll
; ModuleID = 'LLVMDialectModule'
source_filename = "LLVMDialectModule"

declare i8* @malloc(i64)

declare void @free(i8*)

define void @gemm_32(float %0, float %1, float* %2, float* %3, float* %4) !dbg !3 {
  %6 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } undef, float* %2, 0, !dbg !7
  %7 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %6, float* %2, 1, !dbg !9
  %8 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %7, i64 0, 2, !dbg !10
  %9 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %8, i64 32, 3, 0, !dbg !11
  %10 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %9, i64 32, 4, 0, !dbg !12
  %11 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %10, i64 32, 3, 1, !dbg !13
  %12 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %11, i64 1, 4, 1, !dbg !14
  %13 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } undef, float* %3, 0, !dbg !15
  %14 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %13, float* %3, 1, !dbg !16
  %15 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %14, i64 0, 2, !dbg !17
  %16 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %15, i64 32, 3, 0, !dbg !18
  %17 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %16, i64 32, 4, 0, !dbg !19
  %18 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %17, i64 32, 3, 1, !dbg !20
  %19 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %18, i64 1, 4, 1, !dbg !21
  %20 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } undef, float* %4, 0, !dbg !22
  %21 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %20, float* %4, 1, !dbg !23
  %22 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %21, i64 0, 2, !dbg !24
  %23 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %22, i64 32, 3, 0, !dbg !25
  %24 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %23, i64 32, 4, 0, !dbg !26
  %25 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %24, i64 32, 3, 1, !dbg !27
  %26 = insertvalue { float*, float*, i64, [2 x i64], [2 x i64] } %25, i64 1, 4, 1, !dbg !28
  br label %27, !dbg !29

27:                                               ; preds = %74, %5
  %28 = phi i64 [ %75, %74 ], [ 0, %5 ]
  %29 = icmp slt i64 %28, 32, !dbg !30
  br i1 %29, label %30, label %76, !dbg !31

30:                                               ; preds = %27
  br label %31, !dbg !32

31:                                               ; preds = %72, %30
  %32 = phi i64 [ %73, %72 ], [ 0, %30 ]
  %33 = icmp slt i64 %32, 32, !dbg !33
  br i1 %33, label %34, label %74, !dbg !34

34:                                               ; preds = %31
  %35 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %12, 1, !dbg !35
  %36 = mul i64 %28, 32, !dbg !36
  %37 = add i64 %36, %32, !dbg !37
  %38 = getelementptr float, float* %35, i64 %37, !dbg !38
  %39 = load float, float* %38, align 4, !dbg !39
  %40 = fmul float %1, %39, !dbg !40
  %41 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %12, 1, !dbg !41
  %42 = mul i64 %28, 32, !dbg !42
  %43 = add i64 %42, %32, !dbg !43
  %44 = getelementptr float, float* %41, i64 %43, !dbg !44
  store float %40, float* %44, align 4, !dbg !45
  br label %45, !dbg !46

45:                                               ; preds = %48, %34
  %46 = phi i64 [ %71, %48 ], [ 0, %34 ]
  %47 = icmp slt i64 %46, 32, !dbg !47
  br i1 %47, label %48, label %72, !dbg !48

48:                                               ; preds = %45
  %49 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %19, 1, !dbg !49
  %50 = mul i64 %28, 32, !dbg !50
  %51 = add i64 %50, %46, !dbg !51
  %52 = getelementptr float, float* %49, i64 %51, !dbg !52
  %53 = load float, float* %52, align 4, !dbg !53
  %54 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %26, 1, !dbg !54
  %55 = mul i64 %46, 32, !dbg !55
  %56 = add i64 %55, %32, !dbg !56
  %57 = getelementptr float, float* %54, i64 %56, !dbg !57
  %58 = load float, float* %57, align 4, !dbg !58
  %59 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %12, 1, !dbg !59
  %60 = mul i64 %28, 32, !dbg !60
  %61 = add i64 %60, %32, !dbg !61
  %62 = getelementptr float, float* %59, i64 %61, !dbg !62
  %63 = load float, float* %62, align 4, !dbg !63
  %64 = fmul float %0, %53, !dbg !64
  %65 = fmul float %64, %58, !dbg !65
  %66 = fadd float %63, %65, !dbg !66
  %67 = extractvalue { float*, float*, i64, [2 x i64], [2 x i64] } %12, 1, !dbg !67
  %68 = mul i64 %28, 32, !dbg !68
  %69 = add i64 %68, %32, !dbg !69
  %70 = getelementptr float, float* %67, i64 %69, !dbg !70
  store float %66, float* %70, align 4, !dbg !71
  %71 = add i64 %46, 1, !dbg !72
  br label %45, !dbg !73

72:                                               ; preds = %45
  %73 = add i64 %32, 1, !dbg !74
  br label %31, !dbg !75

74:                                               ; preds = %31
  %75 = add i64 %28, 1, !dbg !76
  br label %27, !dbg !77

76:                                               ; preds = %27
  ret void, !dbg !78
}

!llvm.dbg.cu = !{!0}
!llvm.module.flags = !{!2}

!0 = distinct !DICompileUnit(language: DW_LANG_C, file: !1, producer: "mlir", isOptimized: true, runtimeVersion: 0, emissionKind: FullDebug)
!1 = !DIFile(filename: "LLVMDialectModule", directory: "/")
!2 = !{i32 2, !"Debug Info Version", i32 3}
!3 = distinct !DISubprogram(name: "gemm_32", linkageName: "gemm_32", scope: null, file: !4, line: 2, type: !5, scopeLine: 2, spFlags: DISPFlagDefinition | DISPFlagOptimized, unit: !0, retainedNodes: !6)
!4 = !DIFile(filename: "original-gemm_32-ll.mlir", directory: "output")
!5 = !DISubroutineType(types: !6)
!6 = !{}
!7 = !DILocation(line: 4, column: 10, scope: !8)
!8 = !DILexicalBlockFile(scope: !3, file: !4, discriminator: 0)
!9 = !DILocation(line: 5, column: 10, scope: !8)
!10 = !DILocation(line: 7, column: 10, scope: !8)
!11 = !DILocation(line: 9, column: 10, scope: !8)
!12 = !DILocation(line: 11, column: 10, scope: !8)
!13 = !DILocation(line: 13, column: 11, scope: !8)
!14 = !DILocation(line: 15, column: 11, scope: !8)
!15 = !DILocation(line: 17, column: 11, scope: !8)
!16 = !DILocation(line: 18, column: 11, scope: !8)
!17 = !DILocation(line: 20, column: 11, scope: !8)
!18 = !DILocation(line: 22, column: 11, scope: !8)
!19 = !DILocation(line: 24, column: 11, scope: !8)
!20 = !DILocation(line: 26, column: 11, scope: !8)
!21 = !DILocation(line: 28, column: 11, scope: !8)
!22 = !DILocation(line: 30, column: 11, scope: !8)
!23 = !DILocation(line: 31, column: 11, scope: !8)
!24 = !DILocation(line: 33, column: 11, scope: !8)
!25 = !DILocation(line: 35, column: 11, scope: !8)
!26 = !DILocation(line: 37, column: 11, scope: !8)
!27 = !DILocation(line: 39, column: 11, scope: !8)
!28 = !DILocation(line: 41, column: 11, scope: !8)
!29 = !DILocation(line: 45, column: 5, scope: !8)
!30 = !DILocation(line: 47, column: 11, scope: !8)
!31 = !DILocation(line: 48, column: 5, scope: !8)
!32 = !DILocation(line: 53, column: 5, scope: !8)
!33 = !DILocation(line: 55, column: 11, scope: !8)
!34 = !DILocation(line: 56, column: 5, scope: !8)
!35 = !DILocation(line: 58, column: 11, scope: !8)
!36 = !DILocation(line: 60, column: 11, scope: !8)
!37 = !DILocation(line: 61, column: 11, scope: !8)
!38 = !DILocation(line: 62, column: 11, scope: !8)
!39 = !DILocation(line: 63, column: 11, scope: !8)
!40 = !DILocation(line: 64, column: 11, scope: !8)
!41 = !DILocation(line: 65, column: 11, scope: !8)
!42 = !DILocation(line: 67, column: 11, scope: !8)
!43 = !DILocation(line: 68, column: 11, scope: !8)
!44 = !DILocation(line: 69, column: 11, scope: !8)
!45 = !DILocation(line: 70, column: 5, scope: !8)
!46 = !DILocation(line: 74, column: 5, scope: !8)
!47 = !DILocation(line: 76, column: 11, scope: !8)
!48 = !DILocation(line: 77, column: 5, scope: !8)
!49 = !DILocation(line: 79, column: 11, scope: !8)
!50 = !DILocation(line: 81, column: 11, scope: !8)
!51 = !DILocation(line: 82, column: 11, scope: !8)
!52 = !DILocation(line: 83, column: 11, scope: !8)
!53 = !DILocation(line: 84, column: 11, scope: !8)
!54 = !DILocation(line: 85, column: 11, scope: !8)
!55 = !DILocation(line: 87, column: 11, scope: !8)
!56 = !DILocation(line: 88, column: 11, scope: !8)
!57 = !DILocation(line: 89, column: 11, scope: !8)
!58 = !DILocation(line: 90, column: 11, scope: !8)
!59 = !DILocation(line: 91, column: 11, scope: !8)
!60 = !DILocation(line: 93, column: 11, scope: !8)
!61 = !DILocation(line: 94, column: 11, scope: !8)
!62 = !DILocation(line: 95, column: 11, scope: !8)
!63 = !DILocation(line: 96, column: 11, scope: !8)
!64 = !DILocation(line: 97, column: 11, scope: !8)
!65 = !DILocation(line: 98, column: 11, scope: !8)
!66 = !DILocation(line: 99, column: 11, scope: !8)
!67 = !DILocation(line: 100, column: 11, scope: !8)
!68 = !DILocation(line: 102, column: 11, scope: !8)
!69 = !DILocation(line: 103, column: 11, scope: !8)
!70 = !DILocation(line: 104, column: 11, scope: !8)
!71 = !DILocation(line: 105, column: 5, scope: !8)
!72 = !DILocation(line: 106, column: 11, scope: !8)
!73 = !DILocation(line: 107, column: 5, scope: !8)
!74 = !DILocation(line: 109, column: 11, scope: !8)
!75 = !DILocation(line: 110, column: 5, scope: !8)
!76 = !DILocation(line: 112, column: 11, scope: !8)
!77 = !DILocation(line: 113, column: 5, scope: !8)
!78 = !DILocation(line: 115, column: 5, scope: !8)

Writing gemm_32.ll


Prepare the interface specification file

In [ ]:
%%writefile gemm_32.c.interface.xml
<?xml version="1.0"?>
<module>
  <function id="gemm_32">
    <arg id="P0" interface_type="default" interface_typename="float" interface_typename_orig="float" interface_typename_include=""/>
    <arg id="P1" interface_type="default" interface_typename="float" interface_typename_orig="float" interface_typename_include=""/>
    <arg id="P2" interface_type="array" interface_typename="float *" interface_typename_orig="float *" size="1024" interface_typename_include=""/>
    <arg id="P3" interface_type="array" interface_typename="float *" interface_typename_orig="float *" size="1024" interface_typename_include=""/>
    <arg id="P4" interface_type="array" interface_typename="float *" interface_typename_orig="float *" size="1024" interface_typename_include=""/>
  </function>
</module>

Writing gemm_32.c.interface.xml


## Prepare the test.xml for testbench generation

In [ ]:
%%writefile test.xml
<?xml version="1.0"?>
<function>
<testbench 
P0="0.53" 
P1="0.14" 
P2="{0.68, 0.15, 0.13, 0.34, 0.41, 0.39, 0.54, 0.46, 0.13, 0.20, 0.43, 0.67, 0.19, 0.21, 0.88, 0.22, 0.38, 0.82, 0.35, 0.45, 0.30, 0.76, 0.08, 0.01, 0.50, 0.14, 0.67, 0.16, 0.85, 0.42, 0.89, 0.48, 0.19, 0.53, 0.13, 0.95, 0.47, 0.80, 0.11, 0.96, 0.75, 0.28, 0.98, 0.39, 0.19, 0.88, 0.46, 0.54, 0.29, 0.58, 0.88, 0.10, 0.32, 0.99, 0.85, 0.15, 0.90, 0.94, 0.14, 0.46, 0.56, 0.90, 0.35, 0.13, 0.59, 0.37, 0.45, 0.18, 0.82, 0.70, 0.21, 0.39, 0.56, 0.35, 0.84, 0.45, 0.24, 0.92, 0.63, 0.59, 0.91, 0.95, 0.05, 0.25, 0.83, 0.06, 0.65, 0.05, 0.94, 0.72, 0.56, 0.16, 0.12, 0.35, 0.31, 0.34, 0.37, 0.91, 0.13, 0.85, 0.95, 0.58, 0.24, 0.47, 0.61, 0.57, 0.02, 0.24, 0.55, 0.36, 0.09, 0.43, 0.29, 0.50, 0.17, 0.26, 0.52, 0.59, 0.95, 0.40, 0.63, 0.77, 0.42, 0.24, 0.47, 0.65, 0.55, 0.30, 0.96, 0.32, 0.24, 0.21, 0.67, 0.41, 0.54, 0.90, 0.80, 0.50, 0.92, 0.16, 0.74, 0.59, 0.19, 0.23, 0.19, 0.99, 0.09, 0.20, 0.89, 0.53, 0.69, 0.43, 0.42, 0.54, 0.81, 0.05, 0.21, 0.04, 0.73, 0.51, 0.13, 0.20, 0.34, 0.25, 0.30, 0.46, 0.83, 0.97, 0.22, 0.76, 0.15, 0.18, 0.09, 0.09, 0.71, 0.05, 0.58, 0.98, 0.59, 0.02, 0.19, 0.27, 0.20, 0.06, 0.79, 0.70, 0.77, 0.45, 0.72, 0.54, 0.94, 0.91, 0.03, 0.84, 0.86, 0.44, 0.09, 0.76, 0.29, 0.56, 0.83, 0.02, 0.44, 0.70, 0.62, 0.21, 0.75, 0.79, 0.43, 0.31, 0.85, 0.41, 0.73, 0.39, 0.84, 0.28, 0.63, 0.98, 0.43, 0.22, 0.32, 0.79, 0.82, 0.17, 0.43, 0.55, 0.92, 0.45, 0.54, 0.79, 0.02, 0.03, 0.29, 0.15, 0.44, 0.56, 0.74, 0.50, 0.78, 0.88, 0.40, 0.93, 0.82, 0.85, 0.04, 0.72, 0.11, 0.34, 0.24, 0.34, 0.27, 0.40, 0.97, 0.83, 0.28, 0.44, 0.98, 0.72, 0.09, 0.99, 0.80, 0.32, 0.43, 0.92, 0.73, 0.06, 0.50, 0.57, 0.49, 0.07, 0.41, 0.92, 0.70, 0.93, 0.98, 0.09, 0.52, 0.13, 0.97, 0.64, 0.63, 0.94, 0.30, 0.85, 0.63, 0.02, 0.52, 0.20, 0.30, 0.61, 0.42, 0.37, 0.54, 0.74, 0.84, 0.35, 0.23, 0.95, 0.86, 0.31, 0.01, 0.69, 0.63, 0.02, 0.27, 0.31, 0.97, 0.78, 0.77, 0.39, 0.99, 0.31, 0.85, 0.56, 0.92, 0.86, 0.63, 0.63, 0.51, 0.66, 0.81, 0.13, 0.56, 0.08, 0.60, 0.77, 0.37, 0.32, 0.43, 0.62, 0.90, 0.54, 0.67, 0.39, 0.85, 0.87, 0.92, 0.44, 0.46, 0.18, 0.03, 0.05, 0.32, 0.45, 0.08, 0.09, 0.52, 0.47, 0.07, 0.90, 0.71, 0.67, 0.29, 0.96, 0.27, 0.91, 0.07, 0.91, 0.49, 0.23, 0.04, 0.78, 0.92, 0.87, 0.93, 0.15, 0.20, 0.37, 0.76, 0.62, 0.97, 0.30, 0.22, 0.21, 0.31, 0.66, 0.31, 0.04, 0.50, 0.41, 0.34, 0.54, 0.22, 0.66, 0.60, 0.70, 0.81, 0.51, 0.65, 0.21, 0.04, 0.18, 0.34, 0.11, 0.36, 0.52, 0.30, 0.32, 0.27, 0.41, 0.33, 0.66, 0.30, 0.31, 0.04, 0.85, 0.11, 0.35, 0.41, 0.06, 0.25, 0.44, 0.20, 0.37, 0.82, 0.04, 0.97, 0.66, 0.21, 0.42, 0.98, 0.93, 0.02, 0.31, 0.40, 0.77, 0.09, 0.59, 0.32, 0.59, 0.86, 0.60, 0.76, 0.69, 0.53, 0.54, 0.67, 0.25, 0.30, 0.05, 0.84, 0.38, 0.53, 0.50, 0.68, 0.76, 0.69, 0.77, 0.15, 0.89, 0.40, 0.36, 0.60, 0.99, 0.14, 0.30, 0.57, 0.42, 0.87, 0.15, 0.42, 0.95, 0.63, 0.56, 0.13, 0.26, 0.16, 0.08, 0.28, 0.10, 0.61, 0.09, 0.42, 0.31, 0.73, 0.12, 0.27, 0.34, 0.83, 0.45, 0.64, 0.20, 0.91, 0.12, 0.49, 0.12, 0.11, 0.93, 0.23, 0.63, 0.31, 0.86, 0.76, 0.73, 0.04, 0.54, 0.50, 0.28, 0.32, 0.09, 0.65, 0.47, 0.76, 0.59, 0.01, 0.55, 0.71, 0.98, 0.80, 0.39, 0.88, 0.15, 0.24, 0.95, 0.84, 0.42, 0.57, 0.53, 0.26, 0.73, 0.19, 0.66, 0.51, 0.09, 0.55, 0.68, 0.40, 0.70, 0.47, 0.69, 0.25, 0.79, 0.99, 0.74, 0.33, 0.02, 0.42, 0.43, 0.62, 0.42, 0.58, 0.31, 0.87, 0.42, 0.74, 0.24, 0.09, 0.67, 0.09, 0.96, 0.01, 0.97, 0.88, 0.09, 0.95, 0.74, 0.25, 0.69, 0.10, 0.47, 0.03, 0.19, 0.11, 0.67, 0.57, 0.31, 0.87, 0.56, 0.19, 0.95, 0.17, 0.07, 0.71, 0.11, 0.63, 0.33, 0.74, 0.97, 0.85, 0.53, 0.09, 0.21, 0.11, 0.61, 0.03, 0.87, 0.08, 0.58, 0.37, 0.30, 0.03, 0.25, 0.99, 0.72, 0.06, 0.38, 0.60, 0.11, 0.04, 0.14, 0.24, 0.14, 0.90, 0.78, 0.88, 0.24, 0.43, 0.11, 0.06, 0.11, 0.85, 0.57, 0.78, 0.87, 0.64, 0.83, 0.53, 0.72, 0.71, 0.18, 0.29, 0.18, 0.25, 0.88, 0.27, 0.60, 0.21, 0.07, 0.52, 0.79, 0.90, 0.87, 0.23, 0.61, 0.84, 0.99, 0.38, 0.82, 0.70, 0.20, 0.15, 0.79, 0.45, 0.26, 0.62, 0.87, 0.71, 0.74, 0.16, 0.83, 0.06, 0.26, 0.37, 0.07, 0.56, 0.46, 0.23, 0.13, 0.97, 0.41, 0.03, 0.03, 0.07, 0.46, 0.01, 0.89, 0.86, 0.37, 0.08, 0.21, 0.83, 0.99, 0.62, 0.13, 0.63, 0.03, 0.63, 0.06, 0.55, 0.22, 0.23, 0.40, 0.51, 0.70, 0.42, 0.02, 0.33, 0.82, 0.61, 0.51, 0.56, 0.57, 0.42, 0.07, 0.10, 0.56, 0.96, 0.20, 0.85, 0.95, 0.24, 0.74, 0.03, 0.82, 0.26, 0.69, 0.31, 0.26, 0.87, 0.83, 0.09, 0.11, 0.51, 0.77, 0.76, 0.10, 0.49, 0.82, 0.67, 0.63, 0.73, 0.81, 0.41, 0.42, 0.54, 0.50, 0.82, 0.91, 0.31, 0.99, 0.38, 0.56, 0.45, 0.23, 0.39, 0.44, 0.97, 0.76, 0.61, 0.11, 0.08, 0.51, 0.35, 0.77, 0.98, 0.26, 0.01, 0.99, 0.52, 0.43, 0.63, 0.08, 0.35, 0.74, 0.15, 0.47, 0.79, 0.59, 0.65, 0.08, 0.04, 0.80, 0.77, 0.27, 0.79, 0.72, 0.44, 0.23, 0.80, 0.14, 0.22, 0.24, 0.76, 0.87, 0.82, 0.08, 0.32, 0.91, 0.66, 0.98, 0.63, 0.93, 0.03, 0.43, 0.99, 0.55, 0.65, 0.07, 0.96, 0.03, 0.97, 0.27, 0.27, 0.19, 0.19, 0.38, 0.09, 0.32, 0.53, 0.77, 0.57, 0.32, 0.99, 0.11, 0.42, 0.68, 0.21, 0.75, 0.50, 0.17, 0.09, 0.97, 0.87, 0.84, 0.23, 0.26, 0.03, 0.81, 0.29, 0.41, 0.61, 0.90, 0.88, 0.11, 0.97, 0.93, 0.99, 0.01, 0.28, 0.13, 0.98, 0.69, 0.12, 0.14, 0.86, 0.39, 0.89, 0.15, 0.61, 0.14, 0.45, 0.31, 0.57, 0.90, 0.58, 0.48, 0.17, 0.70, 0.84, 0.99, 0.93, 0.78, 0.94, 0.05, 0.21, 0.92, 0.51, 0.04, 0.02, 0.97, 0.91, 0.49, 0.34, 0.92, 0.08, 0.28, 0.38, 0.99, 0.65, 0.08, 0.13, 0.88, 0.21, 0.54, 0.54, 0.04, 0.39, 0.89, 0.46, 0.18, 0.77, 0.14, 0.33, 0.10, 0.48, 0.26, 0.83, 0.59, 0.01, 0.53, 0.16, 0.25, 0.73, 0.10, 0.97, 0.99, 0.23, 0.59, 0.05, 0.65, 0.09, 0.35, 0.65, 0.03, 0.45, 0.20, 0.32, 0.19, 0.91, 0.28, 0.54, 0.96, 0.88, 0.59, 0.79, 0.50, 0.40, 0.47, 0.71, 0.27, 0.99, 0.21, 0.30, 0.59, 0.75, 0.09, 0.27, 0.29, 0.25, 0.53, 0.54, 0.61, 0.72, 0.85, 0.08, 0.26, 0.39, 0.88, 0.78, 0.78, 0.52, 0.73, 0.52, 0.87, 0.20, 0.02, 0.98, 0.18, 0.96, 0.20, 0.07, 0.26, 0.63, 0.57, 0.22, 0.85, 0.15, 0.22, 0.29, 0.75, 0.09, 0.51, 0.27, 0.34, 0.54, 0.28, 0.53, 0.94, 0.65, 0.81, 0.34, 0.75, 0.09, 0.48, 0.17, 0.82, 0.60, 0.40, 0.64, 0.82, 0.37, 0.52, 0.83, 0.27, 0.65, 0.92, 0.37, 0.09, 0.35, 0.55, 0.65, 0.07, 0.94, 0.36, 0.96, 0.74, 0.25, 0.93, 0.55, 0.37, 0.07, 0.24, 0.41, 0.53, 0.60, 0.27, 0.28, 0.33, 0.32, 0.93, 0.89, 0.77, 0.08, 0.39, 0.33, 0.96, 0.56, 0.28, 0.72, 0.31, 0.58, 0.33, 0.24, 0.22}" 
P3="{0.38, 0.41, 0.68, 0.86, 0.23, 0.14, 0.35, 0.01, 0.42, 0.20, 0.58, 0.06, 0.42, 0.88, 0.38, 0.94, 0.57, 0.16, 0.37, 0.45, 0.56, 0.75, 0.77, 0.65, 0.99, 0.72, 0.94, 0.98, 0.58, 0.34, 0.58, 0.02, 0.39, 0.10, 0.37, 0.59, 0.40, 0.26, 0.95, 0.67, 0.27, 0.46, 0.50, 0.50, 0.98, 0.18, 0.32, 0.68, 0.06, 0.07, 0.49, 0.34, 0.96, 0.93, 0.33, 0.66, 0.91, 0.28, 0.15, 0.54, 0.59, 0.72, 0.74, 0.39, 0.70, 0.09, 0.10, 0.65, 0.21, 0.82, 0.54, 0.53, 0.16, 0.36, 0.58, 0.62, 0.60, 0.36, 0.95, 0.91, 0.30, 0.18, 0.05, 0.68, 0.11, 0.02, 0.09, 0.48, 0.68, 0.47, 0.30, 0.78, 0.49, 0.80, 0.14, 0.35, 0.78, 0.27, 0.41, 0.48, 0.17, 0.58, 0.48, 0.25, 0.22, 0.17, 0.21, 0.74, 0.87, 0.83, 0.54, 0.11, 0.78, 0.16, 0.99, 0.45, 0.42, 0.85, 0.56, 0.86, 0.48, 0.39, 0.11, 0.29, 0.95, 0.22, 0.02, 0.99, 0.83, 0.62, 0.86, 0.42, 0.30, 0.03, 0.75, 0.91, 0.93, 0.57, 0.53, 0.75, 0.32, 0.56, 0.50, 0.48, 0.50, 0.63, 0.19, 0.21, 0.55, 0.53, 0.62, 0.20, 0.09, 0.48, 0.80, 0.36, 0.56, 0.88, 0.69, 0.57, 0.45, 0.67, 0.21, 0.01, 0.71, 0.40, 0.04, 0.78, 0.29, 0.65, 0.34, 0.90, 0.82, 0.15, 0.77, 0.80, 0.08, 0.13, 0.08, 0.48, 0.45, 0.60, 0.04, 0.20, 0.79, 0.63, 0.68, 0.29, 0.47, 0.02, 0.05, 0.83, 0.92, 0.87, 0.29, 0.57, 0.03, 0.99, 0.29, 0.48, 0.41, 0.53, 0.19, 0.87, 0.45, 0.28, 0.75, 0.26, 0.44, 0.90, 0.66, 0.01, 0.16, 0.32, 0.64, 0.17, 0.40, 0.53, 0.64, 0.46, 0.99, 0.93, 0.88, 0.85, 0.45, 0.68, 0.86, 0.63, 0.94, 0.02, 0.44, 0.26, 0.13, 0.01, 0.75, 0.08, 0.50, 0.68, 0.42, 0.29, 0.40, 0.28, 0.87, 0.02, 0.95, 0.01, 0.26, 0.65, 0.64, 0.23, 0.44, 0.32, 0.06, 0.84, 0.12, 0.13, 0.15, 0.09, 0.93, 0.76, 0.87, 0.61, 0.48, 0.84, 0.09, 0.82, 0.44, 0.99, 0.97, 0.86, 0.93, 0.91, 0.08, 0.89, 0.89, 0.97, 0.45, 0.33, 0.01, 0.88, 0.54, 0.38, 0.01, 0.91, 0.35, 0.57, 0.84, 0.36, 0.67, 0.75, 0.50, 0.95, 0.64, 0.82, 0.72, 0.41, 0.20, 0.37, 0.38, 0.42, 0.47, 0.92, 0.87, 0.19, 0.49, 0.83, 0.29, 0.58, 0.59, 0.81, 0.88, 0.84, 0.48, 0.63, 0.74, 0.93, 0.58, 0.85, 0.28, 0.52, 0.64, 0.29, 0.60, 0.90, 0.98, 0.68, 0.44, 0.05, 0.84, 0.10, 0.24, 0.81, 0.71, 0.69, 0.93, 0.13, 0.15, 0.23, 0.07, 0.08, 0.25, 0.09, 0.37, 0.76, 0.52, 0.75, 0.70, 0.09, 0.15, 0.37, 0.71, 0.08, 0.33, 0.83, 0.46, 0.26, 0.48, 0.82, 0.16, 0.59, 0.38, 0.05, 0.21, 0.36, 0.11, 0.25, 0.31, 0.34, 0.34, 0.33, 0.10, 0.12, 0.97, 0.12, 0.69, 0.72, 0.81, 0.44, 0.92, 0.96, 0.09, 0.37, 0.65, 0.92, 0.57, 0.01, 0.20, 0.22, 0.30, 0.61, 0.54, 0.48, 0.14, 0.74, 0.03, 0.60, 0.07, 0.74, 0.04, 0.28, 0.54, 0.30, 0.96, 0.82, 0.21, 0.11, 0.83, 0.03, 0.74, 0.52, 0.77, 0.40, 0.22, 0.67, 0.52, 0.72, 0.51, 0.54, 0.17, 0.44, 0.72, 0.72, 0.77, 0.49, 0.06, 0.44, 0.42, 0.86, 0.18, 0.60, 0.18, 0.95, 0.52, 0.27, 0.63, 0.45, 0.57, 0.90, 0.82, 0.66, 0.17, 0.96, 0.81, 0.99, 0.99, 0.52, 0.60, 0.39, 0.67, 0.68, 0.34, 0.09, 0.89, 0.41, 0.20, 0.89, 0.64, 0.45, 0.57, 0.58, 0.79, 0.12, 0.52, 0.38, 0.09, 0.62, 0.18, 0.17, 0.15, 0.28, 0.67, 0.80, 0.59, 0.28, 0.42, 0.09, 0.56, 0.81, 0.14, 0.62, 0.41, 0.62, 0.33, 0.26, 0.42, 0.87, 0.87, 0.41, 0.07, 0.28, 0.64, 0.45, 0.37, 0.25, 0.67, 0.30, 0.55, 0.88, 0.22, 0.92, 0.98, 0.03, 0.61, 0.42, 0.41, 0.14, 0.87, 0.56, 0.80, 0.45, 0.17, 0.57, 0.53, 0.17, 0.41, 0.56, 0.09, 0.44, 0.33, 0.50, 0.09, 0.46, 0.24, 0.42, 0.78, 0.15, 0.37, 0.27, 0.65, 0.71, 0.86, 0.69, 0.57, 0.87, 0.84, 0.83, 0.67, 0.18, 0.20, 0.03, 0.41, 0.66, 0.99, 0.06, 0.30, 0.27, 0.07, 0.31, 0.99, 0.48, 0.62, 0.86, 0.05, 0.18, 0.18, 0.16, 0.15, 0.69, 0.05, 0.77, 0.26, 0.16, 0.37, 0.14, 0.92, 0.07, 0.65, 0.48, 0.25, 0.35, 0.67, 0.27, 0.85, 0.62, 0.37, 0.82, 0.83, 0.42, 0.81, 0.54, 0.38, 0.57, 0.47, 0.67, 0.95, 0.24, 0.89, 0.64, 0.34, 0.31, 0.15, 0.54, 0.52, 0.33, 0.15, 0.29, 0.54, 0.41, 0.90, 0.69, 0.82, 0.09, 0.38, 0.17, 0.14, 0.70, 0.88, 0.47, 0.60, 0.45, 0.25, 0.57, 0.40, 0.11, 0.60, 0.37, 0.10, 0.30, 0.71, 0.27, 0.65, 0.73, 0.81, 0.77, 0.63, 0.57, 0.13, 0.21, 0.57, 0.50, 0.59, 0.99, 0.44, 0.06, 0.75, 0.85, 0.20, 0.02, 0.85, 0.24, 0.81, 0.01, 0.96, 0.43, 0.32, 0.14, 0.77, 0.66, 0.07, 0.27, 0.79, 0.12, 0.09, 0.51, 0.53, 0.88, 0.35, 0.40, 0.23, 0.39, 0.30, 0.63, 0.60, 0.48, 0.73, 0.40, 0.65, 0.33, 0.14, 0.52, 0.35, 0.61, 0.27, 0.68, 0.67, 0.54, 0.91, 0.28, 0.40, 0.24, 0.72, 0.41, 0.22, 0.84, 0.80, 0.13, 0.40, 0.72, 0.01, 0.15, 0.05, 0.27, 0.26, 0.26, 0.38, 0.80, 0.25, 0.64, 0.66, 0.46, 0.56, 0.32, 0.64, 0.40, 0.53, 0.60, 0.75, 0.23, 0.05, 0.66, 0.47, 0.23, 0.38, 0.52, 0.63, 0.71, 0.74, 0.57, 0.09, 0.63, 0.35, 0.66, 0.97, 0.86, 0.12, 0.26, 0.26, 0.46, 0.41, 0.78, 0.23, 0.85, 0.13, 0.23, 0.44, 0.11, 0.13, 0.94, 0.24, 0.55, 0.22, 0.83, 0.05, 0.67, 0.20, 0.05, 0.22, 0.43, 0.58, 0.65, 0.87, 0.62, 0.90, 0.85, 0.78, 0.31, 0.74, 0.45, 0.21, 0.23, 0.59, 0.03, 0.22, 0.27, 0.56, 0.81, 0.80, 0.67, 0.13, 0.07, 0.39, 0.42, 0.80, 0.29, 0.99, 0.70, 0.54, 0.39, 0.15, 0.18, 0.33, 0.89, 0.16, 0.48, 0.61, 0.41, 0.45, 0.20, 0.10, 0.25, 0.21, 0.68, 0.98, 0.73, 0.22, 0.80, 0.16, 0.10, 0.70, 0.73, 0.09, 0.77, 0.84, 0.16, 0.28, 0.54, 0.13, 0.62, 0.82, 0.78, 0.80, 0.60, 0.79, 0.54, 0.21, 0.02, 0.69, 0.06, 0.72, 0.84, 0.66, 0.09, 0.11, 0.65, 0.16, 0.47, 0.27, 0.60, 0.93, 0.72, 0.77, 0.13, 0.79, 0.89, 0.34, 0.91, 0.78, 0.57, 0.26, 0.46, 0.91, 0.44, 0.33, 0.41, 0.84, 0.14, 0.48, 0.35, 0.43, 0.27, 0.24, 0.77, 0.09, 0.10, 0.34, 0.77, 0.63, 0.22, 0.51, 0.77, 0.47, 0.72, 0.94, 0.97, 0.41, 0.63, 0.90, 0.09, 0.54, 0.36, 0.86, 0.20, 0.58, 0.08, 0.97, 0.64, 0.98, 0.83, 0.84, 0.76, 0.77, 0.96, 0.96, 0.96, 0.36, 0.17, 0.61, 0.12, 0.56, 0.88, 0.21, 0.34, 0.62, 0.55, 0.72, 0.23, 0.53, 0.88, 0.71, 0.48, 0.80, 0.73, 0.20, 0.78, 0.38, 0.09, 0.42, 0.70, 0.55, 0.96, 0.62, 0.99, 0.45, 0.08, 0.84, 0.22, 0.16, 0.19, 0.53, 0.59, 0.25, 0.07, 0.34, 0.36, 0.28, 0.76, 0.43, 0.15, 0.44, 0.45, 0.06, 0.95, 0.25, 0.01, 0.06, 0.77, 0.60, 0.28, 0.43, 0.44, 0.48, 0.05, 0.04, 0.63, 0.24, 0.09, 0.74, 0.24, 0.09, 0.73, 0.09, 0.35, 0.06, 0.23, 0.83, 0.31, 0.78, 0.29, 0.79, 0.53, 0.01, 0.55, 0.59, 0.13, 0.30, 0.88, 0.41, 0.30, 0.32, 0.08, 0.84, 0.69, 0.92, 0.75, 0.87, 0.25, 0.80, 0.02, 0.61, 0.99, 0.71, 0.30, 0.36, 0.14, 0.25, 0.48, 0.94, 0.73, 0.84, 0.16, 0.61, 0.38, 0.32, 0.88, 0.53, 0.55, 0.30, 0.89, 0.45, 0.97, 0.84, 0.98, 0.94, 0.78, 0.66, 0.95, 0.79, 0.56, 0.96, 0.24, 0.64, 0.18, 0.79, 0.08, 0.27, 0.05, 0.43, 0.54, 0.51, 0.64, 0.80, 0.87, 0.77, 0.40}" 
P4="{0.43, 0.63, 0.57, 0.92, 0.20, 0.93, 0.43, 0.81, 0.07, 0.77, 0.77, 0.38, 0.73, 0.30, 0.76, 0.75, 0.04, 0.42, 0.71, 0.03, 0.32, 0.54, 0.07, 0.44, 0.44, 0.94, 0.15, 0.58, 0.72, 0.94, 0.07, 0.83, 0.07, 0.17, 0.88, 0.17, 0.60, 0.21, 0.02, 0.88, 0.80, 0.07, 0.96, 0.98, 0.03, 0.13, 0.76, 0.04, 0.33, 0.48, 0.01, 0.05, 0.77, 0.97, 0.23, 0.54, 0.22, 0.76, 0.67, 0.78, 0.97, 0.25, 0.39, 0.35, 0.09, 0.81, 0.77, 0.79, 0.50, 0.48, 0.34, 0.91, 0.57, 0.61, 0.27, 0.79, 0.62, 0.12, 0.64, 0.49, 0.95, 0.14, 0.36, 0.74, 0.33, 0.58, 0.85, 0.61, 0.27, 0.60, 0.67, 0.61, 0.74, 0.13, 0.05, 0.93, 0.95, 0.47, 0.33, 0.45, 0.55, 0.46, 0.87, 0.84, 0.96, 0.09, 0.38, 0.46, 0.11, 0.34, 0.94, 0.02, 0.08, 0.90, 0.97, 0.53, 0.98, 0.26, 0.52, 0.71, 0.86, 0.81, 0.85, 0.02, 0.88, 0.13, 0.71, 0.68, 0.11, 0.73, 0.86, 0.82, 0.46, 0.24, 0.92, 0.99, 0.78, 0.80, 0.67, 0.86, 0.12, 0.33, 0.99, 0.33, 0.78, 0.11, 0.34, 0.53, 0.51, 0.09, 0.56, 0.15, 0.37, 0.88, 0.21, 0.08, 0.71, 0.71, 0.21, 0.61, 0.08, 0.40, 0.49, 0.26, 0.42, 0.14, 0.31, 0.32, 0.76, 0.83, 0.47, 0.85, 0.97, 0.15, 0.08, 0.11, 0.09, 0.83, 0.40, 0.42, 0.53, 0.27, 0.84, 0.30, 0.66, 0.32, 0.65, 0.74, 0.20, 0.93, 0.27, 0.59, 0.41, 0.05, 0.15, 0.61, 0.86, 0.39, 0.09, 0.92, 0.57, 0.89, 0.32, 0.72, 0.09, 0.41, 0.27, 0.10, 0.32, 0.18, 0.67, 0.05, 0.70, 0.45, 0.65, 0.02, 0.82, 0.66, 0.84, 0.30, 0.42, 0.09, 0.86, 0.31, 0.83, 0.09, 0.76, 0.09, 0.32, 0.47, 0.97, 0.45, 0.92, 0.28, 0.69, 0.53, 0.31, 0.88, 0.18, 0.31, 0.17, 0.68, 0.78, 0.09, 0.31, 0.49, 0.52, 0.60, 0.01, 0.86, 0.02, 0.14, 0.48, 0.38, 0.99, 0.51, 0.78, 0.34, 0.57, 0.39, 0.79, 0.01, 0.82, 0.07, 0.40, 0.59, 0.07, 0.42, 0.28, 0.49, 0.28, 0.21, 0.03, 0.35, 0.40, 0.45, 0.94, 0.46, 0.69, 0.78, 0.23, 0.32, 0.78, 0.93, 0.75, 0.21, 0.86, 0.75, 0.80, 0.06, 0.77, 0.99, 0.04, 0.74, 0.84, 0.93, 0.76, 0.95, 0.75, 0.18, 0.63, 0.10, 0.57, 0.25, 0.09, 0.01, 0.14, 0.54, 0.40, 0.84, 0.27, 0.72, 0.61, 0.40, 0.45, 0.90, 0.53, 0.18, 0.55, 0.35, 0.11, 0.64, 0.71, 0.80, 0.48, 0.48, 0.06, 0.41, 0.88, 0.65, 0.28, 0.58, 0.09, 0.83, 0.30, 0.69, 0.87, 0.09, 0.10, 0.45, 0.27, 0.81, 0.71, 0.39, 0.20, 0.79, 0.09, 0.79, 0.51, 0.47, 0.06, 0.43, 0.58, 0.41, 0.27, 0.30, 0.78, 0.61, 0.33, 0.66, 0.40, 0.37, 0.38, 0.60, 0.09, 0.32, 0.61, 0.51, 0.63, 0.09, 0.09, 0.10, 0.24, 0.63, 0.80, 0.78, 0.52, 0.52, 0.90, 0.19, 0.81, 0.26, 0.99, 0.07, 0.48, 0.80, 0.16, 0.32, 0.94, 0.78, 0.36, 0.45, 0.32, 0.87, 0.86, 0.19, 0.22, 0.96, 0.33, 0.11, 0.90, 0.98, 0.85, 0.39, 0.89, 0.51, 0.27, 0.10, 0.94, 0.72, 0.78, 0.41, 0.54, 0.42, 0.77, 0.83, 0.94, 0.21, 0.08, 0.51, 0.62, 0.60, 0.25, 0.52, 0.37, 0.43, 0.64, 0.21, 0.47, 0.54, 0.67, 0.65, 0.09, 0.37, 0.71, 0.81, 0.53, 0.89, 0.35, 0.75, 0.71, 0.83, 0.57, 0.61, 0.38, 0.13, 0.02, 0.97, 0.45, 0.16, 0.17, 0.26, 0.99, 0.32, 0.04, 0.64, 0.22, 0.63, 0.12, 0.84, 0.59, 0.84, 0.04, 0.17, 0.79, 0.55, 0.94, 0.16, 0.14, 0.52, 0.25, 0.37, 0.08, 0.57, 0.40, 0.17, 0.44, 0.33, 0.66, 0.79, 0.63, 0.54, 0.76, 0.34, 0.78, 0.36, 0.07, 0.80, 0.75, 0.88, 0.06, 0.84, 0.69, 0.08, 0.40, 0.68, 0.23, 0.80, 0.51, 0.38, 0.57, 0.19, 0.38, 0.19, 0.13, 0.10, 0.83, 0.32, 0.75, 0.63, 0.69, 0.47, 0.04, 0.61, 0.09, 0.51, 0.99, 0.89, 0.12, 0.80, 0.49, 0.61, 0.29, 0.59, 0.70, 0.03, 0.44, 0.91, 0.70, 0.90, 0.67, 0.90, 0.23, 0.51, 0.81, 0.20, 0.29, 0.78, 0.05, 0.85, 0.33, 0.44, 0.24, 0.17, 0.01, 0.15, 0.21, 0.53, 0.74, 0.17, 0.56, 0.32, 0.16, 0.30, 0.71, 0.19, 0.72, 0.35, 0.52, 0.78, 0.12, 0.39, 0.09, 0.70, 0.61, 0.54, 0.33, 0.57, 0.91, 0.72, 0.91, 0.98, 0.04, 0.34, 0.79, 0.91, 0.86, 0.89, 0.58, 0.20, 0.87, 0.85, 0.47, 0.94, 0.97, 0.20, 0.18, 0.09, 0.95, 0.28, 0.70, 0.29, 0.96, 0.80, 0.52, 0.33, 0.75, 0.11, 0.26, 0.64, 0.06, 0.91, 0.31, 0.81, 0.21, 0.01, 0.38, 0.31, 0.96, 0.66, 0.28, 0.83, 0.19, 0.67, 0.65, 0.78, 0.52, 0.12, 0.66, 0.50, 0.15, 0.99, 0.13, 0.79, 0.17, 0.69, 0.57, 0.15, 0.68, 0.27, 0.23, 0.34, 0.37, 0.31, 0.39, 0.31, 0.69, 0.49, 0.83, 0.49, 0.83, 0.15, 0.82, 0.94, 0.48, 0.92, 0.69, 0.70, 0.17, 0.06, 0.53, 0.70, 0.99, 0.23, 0.92, 0.63, 0.98, 0.19, 0.77, 0.96, 0.95, 0.43, 0.02, 0.99, 0.44, 0.51, 0.93, 0.92, 0.60, 0.16, 0.56, 0.77, 0.31, 0.29, 0.30, 0.93, 0.82, 0.44, 0.74, 0.21, 0.26, 0.25, 0.76, 0.45, 0.09, 0.39, 0.41, 0.96, 0.37, 0.61, 0.50, 0.91, 0.86, 0.57, 0.94, 0.05, 0.34, 0.19, 0.14, 0.96, 0.03, 0.75, 0.47, 0.37, 0.57, 0.19, 0.22, 0.65, 0.31, 0.74, 0.69, 0.78, 0.20, 0.28, 0.44, 0.52, 0.88, 0.13, 0.52, 0.58, 0.68, 0.21, 0.17, 0.52, 0.04, 0.77, 0.92, 0.15, 0.82, 0.87, 0.34, 0.25, 0.25, 0.31, 0.06, 0.36, 0.17, 0.70, 0.39, 0.78, 0.04, 0.01, 0.04, 0.90, 0.46, 0.67, 0.39, 0.12, 0.25, 0.67, 0.89, 0.29, 0.98, 0.67, 0.71, 0.84, 0.16, 0.25, 0.82, 0.14, 0.29, 0.14, 0.70, 0.05, 0.44, 0.99, 0.91, 0.72, 0.54, 0.98, 0.37, 0.58, 0.68, 0.68, 0.26, 0.50, 0.11, 0.26, 0.13, 0.84, 0.32, 0.90, 0.75, 0.92, 0.18, 0.62, 0.89, 0.45, 0.79, 0.26, 0.14, 0.95, 0.72, 0.37, 0.93, 0.01, 0.14, 0.77, 0.89, 0.96, 0.75, 0.82, 0.96, 0.10, 0.90, 0.77, 0.31, 0.07, 0.44, 0.76, 0.77, 0.33, 0.86, 0.07, 0.74, 0.91, 0.35, 0.86, 0.39, 0.27, 0.70, 0.35, 0.56, 0.65, 0.86, 0.18, 0.97, 0.60, 0.11, 0.51, 0.09, 0.14, 0.46, 0.09, 0.98, 0.59, 0.37, 0.83, 0.98, 0.56, 0.53, 0.62, 0.80, 0.26, 0.25, 0.62, 0.88, 0.26, 0.64, 0.49, 0.81, 0.64, 0.17, 0.25, 0.67, 0.10, 0.24, 0.70, 0.32, 0.53, 0.34, 0.76, 0.18, 0.91, 0.59, 0.17, 0.22, 0.67, 0.19, 0.52, 0.76, 0.89, 0.50, 0.45, 0.48, 0.87, 0.14, 0.31, 0.28, 0.97, 0.90, 0.28, 0.40, 0.18, 0.77, 0.30, 0.14, 0.90, 0.91, 0.30, 0.72, 0.01, 0.05, 0.91, 0.17, 0.04, 0.11, 0.03, 0.84, 0.49, 0.92, 0.84, 0.54, 0.07, 0.39, 0.53, 0.08, 0.31, 0.72, 0.31, 0.17, 0.61, 0.23, 0.17, 0.41, 0.90, 0.74, 0.21, 0.09, 0.77, 0.13, 0.83, 0.90, 0.37, 0.75, 0.93, 0.56, 0.63, 0.19, 0.25, 0.92, 0.43, 0.04, 0.01, 0.25, 0.23, 0.74, 0.67, 0.70, 0.43, 0.84, 0.06, 0.82, 0.63, 0.15, 0.75, 0.72, 0.01, 0.95, 0.44, 0.28, 0.13, 0.88, 0.84, 0.01, 0.99, 0.94, 0.35, 0.39, 0.98, 0.88, 0.12, 0.90, 0.33, 0.33, 0.38, 0.34, 0.85, 0.53, 0.49, 0.43, 0.05, 0.50, 0.26, 0.77, 0.46, 0.03, 0.03, 0.39, 0.70, 0.28, 0.26, 0.31, 0.13, 0.38, 0.15, 0.57, 0.68, 0.02, 0.83, 0.17, 0.37, 0.49, 0.06, 0.47, 0.36, 0.81, 0.92, 0.16, 0.64, 0.17, 0.87, 0.35, 0.07, 0.83, 0.45, 0.02, 0.41, 0.11, 0.86, 0.78, 0.89, 0.94, 0.90, 0.07, 0.24, 0.36, 0.25, 0.55, 0.29, 0.84, 0.38, 0.36, 0.97}" />
</function>

Writing test.xml


## Run Bambu on the .ll file

In [ ]:
!bambu gemm_32.ll --simulate --no-clean --compiler=I386_CLANG12 --top-fname=gemm_32 --simulator=VERILATOR -v4 --disable-function-proxy --print-dot |& tee log.txt

## Custom floating point synthesis



In [ ]:
!bambu gemm_32.ll --simulate --no-clean --compiler=I386_CLANG12 --top-fname=gemm_32 --simulator=VERILATOR -v4 --disable-function-proxy --print-dot --fp-format="gemm_32*8*7*-127*0*0*1*0*U" --max-ulp=2000000 |& tee log.bfloat16.txt

# SIMD vectorization

Source code: /content/bambu-tutorial/04-simd/Exercise1/histogram.c


Generate an accelerator with vector size of 1.

Bambu script: /content/bambu-tutorial/04-simd/Exercise1/solution/bambu.sh

In [ ]:
%cd /content/bambu-tutorial/04-simd/Exercise1/
!./solution/bambu.sh

Generate an accelerator with vector size of 4.

Bambu script: /content/bambu-tutorial/04-simd/Exercise2/solution/bambu.sh

In [ ]:
%cd /content/bambu-tutorial/04-simd/Exercise2/
!./solution/bambu.sh

/content/bambu-tutorial/04-simd/Exercise2
 ==  Bambu executed with: /tmp/.mount_bambuq5S2AB/usr/bin/bambu --compiler=I386_GCC49 --experimental-setup=BAMBU-BALANCED-MP --device-name=5SGXEA7N2F45C1 --clock-period=10 -fwhole-program -fno-delete-null-pointer-checks -fdisable-tree-cunroll -fdisable-tree-ivopts --param max-inline-insns-auto=1000 -fopenmp-simd=4 --simulate /content/bambu-tutorial/04-simd/Exercise2/solution/../histogram.c 


********************************************************************************
                    ____                  _
                   | __ )  __ _ _ __ ___ | |_   _   _
                   |  _ \ / _` | '_ ` _ \| '_ \| | | |
                   | |_) | (_| | | | | | | |_) | |_| |
                   |____/ \__,_|_| |_| |_|_.__/ \__,_|

********************************************************************************
                         High-Level Synthesis Tool

                         Politecnico di Milano - DEIB
                          Syst

Obtained speedup of 2.93

# OpenMP parallel for

Source code: /content/bambu-tutorial/05-context-switch/Exercise1/trinityq4/lubm_trinityq4.c


Sequential accelerator for the LUBM-t4 benchmark.

Bambu script: /content/bambu-tutorial/05-context-switch/Exercise1/solution/bambu.sh

In [ ]:
%cd /content/bambu-tutorial/05-context-switch/Exercise1/
!./solution/bambu.sh

Parallel accelerator with context switching.
*   2 copies of the kernel are synthesized
*   4 external memory banks with 2 channels
*   4 context switches

Bambu script: /content/bambu-tutorial/05-context-switch/Exercise3/solution/bambu.sh


In [ ]:
%cd /content/bambu-tutorial/05-context-switch/Exercise3/
!./solution/bambu.sh

Obtained speedup 3.33